# Colab & GPU Verification

Run this cell to verify:
1. Connection to Google Colab
2. GPU availability (A100, V100, T4, etc.)
3. CUDA functionality


In [ ]:
# Verification Code - Copy and run this in Colab

import sys
import os

print("="*70)
print("COLAB & GPU VERIFICATION")
print("="*70)

# 1. Verify Colab Connection
print("\n[1] COLAB CONNECTION CHECK")
print("-" * 70)
try:
    import google.colab
    print("✓ Running in Google Colab")
    print(f"✓ Colab module imported successfully")
    colab_connected = True
except ImportError:
    print("✗ NOT running in Google Colab")
    colab_connected = False

print(f"\nCurrent directory: {os.getcwd()}")
if "/content" in os.getcwd():
    print("✓ In Colab filesystem (/content)")

# 2. Verify GPU/CUDA
print("\n[2] GPU/CUDA CHECK")
print("-" * 70)
try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
    
    cuda_available = torch.cuda.is_available()
    print(f"CUDA available: {cuda_available}")
    
    if cuda_available:
        gpu_count = torch.cuda.device_count()
        print(f"Number of GPUs: {gpu_count}")
        
        for i in range(gpu_count):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_props = torch.cuda.get_device_properties(i)
            gpu_memory_gb = gpu_props.total_memory / 1e9
            
            print(f"\n  GPU {i}:")
            print(f"    Name: {gpu_name}")
            print(f"    Memory: {gpu_memory_gb:.2f} GB")
            print(f"    Compute Capability: {gpu_props.major}.{gpu_props.minor}")
            
            if "A100" in gpu_name:
                print(f"    ✓ A100 GPU detected! (Best for training!)")
            elif "V100" in gpu_name:
                print(f"    ✓ V100 GPU detected (Good)")
            elif "T4" in gpu_name:
                print(f"    ✓ T4 GPU detected (Minimum)")
        
        # Test CUDA operation
        test_tensor = torch.randn(10, 10).cuda()
        print(f"\n✓ CUDA operations working (test tensor created on GPU)")
        del test_tensor
        torch.cuda.empty_cache()
    else:
        print("\n✗ NO GPU DETECTED!")
        print("\n  To enable GPU:")
        print("  1. Runtime → Change runtime type")
        print("  2. Set Hardware accelerator: GPU")
        print("  3. Save and restart runtime")
        
except ImportError:
    print("✗ PyTorch not installed")
except Exception as e:
    print(f"✗ Error: {e}")

# Summary
print("\n" + "="*70)
print("SUMMARY")
print("="*70)
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ Colab: Connected")
        print(f"✓ GPU: {gpu_name}")
        if "A100" in gpu_name:
            print("✅ READY! A100 GPU detected - Best for training!")
        else:
            print("✅ READY! GPU available for training")
    else:
        print("✓ Colab: Connected")
        print("✗ GPU: Not available - Enable GPU first!")
except:
    pass
print("="*70)
